# Давай попробуем всякие opensource-ные решения вкрутить

In [1]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "ru"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
SAMPLES = 5

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)


/Users/danilandreev/Desktop/Python/DataScience/asr-cli/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/danilandreev/Desktop/Python/DataScience/asr-cli/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architect

In [2]:
# model.save_pretrained("resources/wav2vec2ctc.pt")
# processor.save_pretrained("resources/wav2vec2ctc_processor.pt")

In [3]:
example_path = "resources/example.wav"
samples, sr = torchaudio.load(example_path)
print(samples.shape, sr)
# Downsampling to 16k
transform = torchaudio.transforms.Resample(
    orig_freq=sr, new_freq=16_000
)
samples = transform(samples).squeeze(0)
sr = 16_000
print(samples.shape, sr)


torch.Size([1, 101376]) 32000
torch.Size([50688]) 16000


In [4]:
inputs = processor(samples, sampling_rate=sr, return_tensors="pt", padding=True)
print(inputs["input_values"].shape)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)
predicted_sentences

torch.Size([1, 50688])


['а и правда никакой']

In [5]:
# нихуя себе, работает)

In [6]:
inputs.input_values.shape

torch.Size([1, 50688])